# Формула Шерман-Моррисона и трехдиагональная матрица

In [23]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt

Разберем пример, уравнение пружинного маятника с внешней силой: 
$$ \ddot x(t) + k^2 x(t) = f(t) $$ 
При этом будем считать функцию $f(t)$ переодичной с периодом $T$. Очевидно, что устоявшееся решение уравнения также должно быть периодично. Ищем решение на интервале $t = (0, T)$.

Для численного расчета ввдем равномерную сетку из $N$ узлов:
$$t_n = \frac{n}{N} T$$
$$x_n = x(t_n) $$
$$f_n = f(t_n) $$
$$\ddot x_n = \frac{x_{n+1} - 2x_n + y_{x-1}}{\Delta t^2} $$





*Задать парметры системы, k, N, T. Определить вектора $t_n$, $f_n$. Например $f = sin(t) + sin(3t) + cos(5t)$*

Уравнение на сетке перепишется в виде:
$$ \frac{x_{n+1} - 2x_n + x_{n-1}}{\Delta t^2} + k^2 x_n = f_n $$
$$ \frac{1}{\Delta t^2}x_{n+1} + \left(k^2 - \frac{2}{\Delta t^2} \right) x_n + \frac{1}{\Delta t^2}x_{n-1} = f_n $$
Или введя обозначения $\alpha = k^2 \Delta t^2 - 2$ и $F_n = f_n \Delta t^2$
$$ x_{n+1} + \alpha x_n + x_{n-1} = F_n $$
для $n = (1,N-2)$.




*Определить $\alpha$ и $F$*

Из граничных условий получаем уравнения для $x_0$ и $x_{N-1}$:
$$ x_{1} + \alpha x_0 + x_{N-1} = F_0 $$
$$ x_{0} + \alpha x_{N-1} + x_{N-2} = F_{N-1} $$


Все вместе это можно записать в матричном виде:
$$ 
\begin{pmatrix} 
    \alpha & 1 & 0 & 0 & \dots & 0 & 0 & 1 \\
    1 & \alpha & 1 & 0 & \dots & 0 & 0 & 0 \\
    0 & 1 & \alpha & 1 & \dots & 0 & 0 & 0 \\
    0 & 0 & 1 & \alpha & \dots & 0 & 0 & 0 \\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
    0 & 0 & 0 & 0 & \dots & \alpha & 1 & 0 \\ 
    0 & 0 & 0 & 0 & \dots & 1 & \alpha & 1 \\ 
    1 & 0 & 0 & 0 & \dots & 0 & 1 & \alpha \\ 
\end{pmatrix} 
\begin{pmatrix} 
    x_0 \\
    x_1 \\
    x_2 \\
    x_3 \\
    \vdots \\
    x_{N-3} \\
    x_{N-2} \\
    x_{N-1} \\
\end{pmatrix}
=
\begin{pmatrix} 
    F_0 \\
    F_1 \\
    F_2 \\
    F_3 \\
    \vdots \\
    F_{N-3} \\
    F_{N-2} \\
    F_{N-1} \\
\end{pmatrix}
$$


Полученная матрица очень похожа на трехдиаганальную (для решения которых существуют очень эффективные методы, см. scipy.linalg.solve_banded). Но она отличается наличием ненулевых элементах в углах матрицы, из-за чего напрямую применить кк ней быстрые методы нельзя.

На помощь приходит формула Шерман-Моррисона. Пусть есть матрица $A$, решение для которой мы можем посчитать быстро. Пусть новая матрица $B = A + {\bf uv}^T$ отличается от известной добавкой, представимой в виде произведения двух векторов ${\bf uv}^T$.

Тогда, чтобы получить решение уравнения
$$ (A + {\bf uv}^T) {\bf x} = {\bf f} $$
достаточно получить $x_0$ и $y$ из уравнений:
$$ A {\bf x}_0 = {\bf f} \qquad A {\bf y} = {\bf u} $$
тогда решение исходного уравнения:
$$ {\bf x} = {\bf x}_0 - \frac{({\bf v} \cdot {\bf x}_0 )}{1 + ({\bf v} \cdot {\bf y})} {\bf y} $$


Несложно получить, что ветора $\bf u$ и $\bf v$ должны иметь вид:
$$
{\bf u} = 
\begin{pmatrix} 
    \gamma \\
    0 \\
    \vdots \\
    0 \\
    1 \\
\end{pmatrix}
\qquad 
{\bf v} = 
\begin{pmatrix} 
    1 \\
    0 \\
    \vdots \\
    0 \\
    1/\gamma \\
\end{pmatrix}
\qquad
{\bf u v}^T = 
\begin{pmatrix} 
    \gamma & 0 & \dots & 0 & 1 \\
    0 & 0 &\dots & 0 & 0 \\
    \vdots & \vdots & \ddots & \vdots & \vdots \\
    0 & 0 & \dots & 0 & 0 \\ 
    1 & 0 & \dots & 0 & 1/\gamma \\ 
\end{pmatrix} 
$$
где $\gamma$ произвольное число.

Тогда матрица $A$ в нашем случае:
$$
A = 
\begin{pmatrix} 
    \alpha - \gamma & 1 & 0 & 0 & \dots & 0 & 0 & 0 \\
    1 & \alpha & 1 & 0 & \dots & 0 & 0 & 0 \\
    0 & 1 & \alpha & 1 & \dots & 0 & 0 & 0 \\
    0 & 0 & 1 & \alpha & \dots & 0 & 0 & 0 \\
    \vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
    0 & 0 & 0 & 0 & \dots & \alpha & 1 & 0 \\ 
    0 & 0 & 0 & 0 & \dots & 1 & \alpha & 1 \\ 
    0 & 0 & 0 & 0 & \dots & 0 & 1 & \alpha - 1/\gamma \\ 
\end{pmatrix} 
$$


*Задать вектора $\bf v$ и $\bf u$*

*Определить диагонали матрицы в виде трехстрочной матрицы (см. документацию solve_banded) *

In [27]:
diag_up = 
diag_mid =
diag_low =

A_band = np.concatenate(([diag_up], [diag_mid], [diag_low]), axis = 0)

*Решить уравнения $A {\bf x}_0 = {\bf f}$ и $ A {\bf y} = {\bf u} $ и найти $\bf x$. Построить графики (t,x) и (t,f). Поварировать параметры задачи. Что будет, если собственная частота $k$ совпадет с одной из частот $f_n$?*